In [1]:
import cbayes.sample
import cbayes.distributions
import cbayes.solve
import numpy as np
import ipywidgets as wd
import matplotlib.pyplot as plt

### Define a function that generates an arbitrarily ill-condidtioned 2-2 map


In [2]:
def make_model(skew):
    # this function makes a linear map whos first component is the x-unit vector
    # and each subsequent component is a norm-1 vector satisfying the property
    # that the 2-2 map made from it and the aforementioned unit vector is a map
    # with skewness in skew_range, which is a list of desired skewnesses   
    # TODO currently this map only works for 2-D input space     
    
    def my_model(parameter_samples):
        Q_map = [ [1.0, 0.0] ] # all map components have the same norm, rect_size to have measures of events equal btwn spaces.
        Q_map.append( [np.sqrt(np.array(skew)**2 - 1), 1] ) # taken with the first component, this leads to a 2-2 map with skewsness 's'
        Q_map = np.array( Q_map )
        QoI_samples = np.dot(parameter_samples, np.transpose(Q_map))
        return QoI_samples
    return my_model

#### Visualize the contours of this vector-valued map.

### Generate samples and map them

In [57]:
def compare(num_samples=5000, skew=1, prior_std = 1.0, data_std = 0.25, color_map = 'jet', num_levels = 40, seed=12):
    obs_data = np.array([0., 0.])
    
    model = make_model([skew])
    mse_fun = cbayes.sample.MSE_generator(model, obs_data, data_std)
    
    
    s_input_set = cbayes.sample.sample_set(size=(num_samples, 2))
    s_input_set.set_dist(dim=0, distribution='normal', kwds={'loc': 0, 'scale': prior_std})
    s_input_set.set_dist(dim=1, distribution='normal', kwds={'loc': 0, 'scale': prior_std})
    input_samples = s_input_set.generate_samples(seed=seed)
    
    output_samples_vector_valued = model(input_samples)
    output_samples_scalar_valued = mse_fun(input_samples).reshape(-1,1)



    # VECTOR PROBLEM
    s_output_set_vector_valued = cbayes.sample.sample_set(size=(num_samples, 2))
    s_output_set_vector_valued.samples = output_samples_vector_valued
    p_set_vector = cbayes.sample.problem_set(s_input_set, s_output_set_vector_valued)
    # Set observed 
    p_set_vector.set_observed_dist(dim=0, dist='normal', kwds={'loc': 0, 'scale': data_std})
    p_set_vector.set_observed_dist(dim=1, dist='normal', kwds={'loc': 0, 'scale': data_std})
    p_set_vector.model = model
    
    p_set_vector.compute_pushforward_dist()
    p_set_vector.set_ratio()
    
    cbayes.solve.problem(p_set_vector)
    accepted_inputs_vector = p_set_vector.input.samples[p_set_vector.accept_inds,:]
    print('num accepted for vector-valued:', len(accepted_inputs_vector), 'mean:', np.mean(accepted_inputs_vector))

    # SCALAR PROBLEM
    s_output_set_scalar_valued = cbayes.sample.sample_set(size=(num_samples, 1))
    s_output_set_scalar_valued.samples = output_samples_scalar_valued
    p_set_scalar = cbayes.sample.problem_set(s_input_set, s_output_set_scalar_valued)
    # Set observed 
    num_observations = 2
    p_set_scalar.set_observed_dist('gamma', {'a':num_observations/2.0, 'scale':2.0/num_observations}, dim=0)
    p_set_scalar.model = mse_fun
    
    p_set_scalar.compute_pushforward_dist()
    p_set_scalar.set_ratio()
    
    cbayes.solve.problem(p_set_scalar)
    accepted_inputs_scalar = p_set_scalar.input.samples[p_set_scalar.accept_inds,:]
    print('num accepted for scalar-valued:', len(accepted_inputs_scalar), 'mean:', np.mean(accepted_inputs_scalar))
    
    # PLOTTING
    x = input_samples[:,0] 
    y = input_samples[:,1]
    xs = accepted_inputs_scalar[:,0]
    ys = accepted_inputs_scalar[:,1]

    xv = accepted_inputs_vector[:,0]
    yv = accepted_inputs_vector[:,1]
    
    
    fig, axs = plt.subplots(ncols=2, nrows=3, figsize=(12,18))
    axs[0,0].tricontour(x, y, output_samples_vector_valued[:,0], num_levels, cmap=color_map)
    axs[0,0].tricontour(x, y, output_samples_vector_valued[:,1], num_levels, cmap=color_map)
    axs[0,1].tricontour(x, y, output_samples_scalar_valued[:,0], num_levels, cmap=color_map)
    
    vmin, vmax = 0, 1
    axs[1,0].tricontourf(x, y, p_set_vector.ratio/np.max(p_set_vector.ratio), int(num_levels/2), cmap=color_map,
                          vmin=vmin, vmax=vmax)
    axs[1,1].tricontourf(x, y, p_set_scalar.ratio/np.max(p_set_scalar.ratio), int(num_levels/2), cmap=color_map,
                          vmin=vmin, vmax=vmax)

    bound_val = 1.0
    axs[0,0].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[0,1].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[1,0].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[1,1].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[0,0].scatter(obs_data[0], obs_data[1], 100, 'k')
    axs[0,1].scatter(obs_data[0], obs_data[1], 100, 'k')
    axs[1,0].scatter(obs_data[0], obs_data[1], 100, 'k')
    axs[1,1].scatter(obs_data[0], obs_data[1], 100, 'k')
    axs[1,0].scatter(xv, yv, color='w', alpha=0.25)
    axs[1,1].scatter(xs, ys, color='w', alpha=0.25)
    
    num_plot_pts = 100
    xx = np.linspace(0, 50, num_plot_pts)
#     axs[2,1].plot(xx, p_set_scalar.pushforward_dist.pdf(xx))
#     axs[2,1].plot(xx, p_set_scalar.observed_dist.pdf(xx))
    
    xxx = np.linspace(-bound_val, bound_val,num_plot_pts)
    XX, YY = np.meshgrid(xxx,xxx)
    lam = np.vstack([XX.ravel(), YY.ravel()]).T
#     print(lam.shape)
    zzs = p_set_scalar.evaluate_posterior(lam)
    zzs = zzs.reshape(num_plot_pts,num_plot_pts)
    line0, = axs[2,0].plot(xxx, np.sum(zzs, axis=0), '-', lw=2)
    line1, = axs[2,0].plot(xxx, np.sum(zzs, axis=1), '--', lw=2)
    plt.legend([line0, line1], ['$\\lambda_1$', '$\\lambda_2$' ])
    
    zzv = p_set_vector.evaluate_posterior(lam)
    zzv = zzv.reshape(num_plot_pts,num_plot_pts)
    line3, = axs[2,1].plot(xxx, np.sum(zzv, axis=0), '-', lw=2)
    line4, = axs[2,1].plot(xxx, np.sum(zzv, axis=1), '--', lw=2)
    plt.legend([line3, line4], ['$\\lambda_1$', '$\\lambda_2$' ])
    
    axs[2,0].vlines(0,0,50, 'k', lw=1)
    axs[2,1].vlines(0,0,50, 'k', lw=1)
    axs[2,1].axis([-bound_val, bound_val, 0, 300])
    axs[2,0].axis([-bound_val, bound_val, 0, 300])
    plt.show()



In [58]:
colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'jet', 
             'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

out = wd.interact(compare, 
            num_samples = wd.IntSlider(value=1000, min=1000, max=10000, step=100, continuous_update=False), 
            skew = wd.FloatSlider(value=1.0, min=1.0, max=4.0, step=0.05, continuous_update=False), 
            prior_std = wd.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.05, continuous_update=False), 
            data_std = wd.FloatSlider(value=0.25, min=0.05, max=1, step=0.05, continuous_update=False), 
            color_map = wd.Dropdown(value='jet', options=colormaps), 
            num_levels = wd.IntSlider(value=40, min=20, max=100, step=5, continuous_update=False),
            seed = wd.IntSlider(value=0, min=0, max=1000, step=1, continuous_update=False) )

interactive(children=(IntSlider(value=1000, continuous_update=False, description='num_samples', max=10000, min…